# Notebook 03: Routing Simulation & Strategy Optimization

**Author:** Hector Carbajal  
**Version:** 1.1  
**Last Updated:** 2026-02

---

## Purpose
This notebook develops and validates fleet routing strategies. We compare the baseline (FCFS) against optimization strategies (Nearest-Driver, Capacity-Aware) to identify the best balance between cost efficiency and service level agreements (SLA).

## Inputs
- `data/processed/trips_with_efficiency.csv` - Trips with efficiency scores
- `data/processed/drivers.csv` - Driver reference data

## Key Strategy Metrics
- **Deadhead Miles**: Miles driven without a passenger.
- **Cost per Trip**: Operational cost based on $2.15/mile benchmark.
- **SLA Breach Risk**: Probability of late pickup under specific load conditions.

## Table of Contents
1. [Setup & Data Verification](#setup)
2. [Strategy Execution](#simulation)
3. [Multi-Metric Performance Comparison](#comparison)
4. [💰 Financial Impact & ROI Projection](#finance)
5. [💡 Key Findings](#findings)

In [1]:
# Setup
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
from pathlib import Path

# Project imports
sys.path.insert(0, str(Path.cwd().parent))
from src.config import RANDOM_SEED, PROCESSED_DIR
from src.routing_simulation import run_simulation_comparison

# Global Aesthetic Config
px.defaults.template = "plotly_white"
STRATEGY_COLORS = {"FCFS": "#94a3b8", "Nearest": "#3b82f6", "Capacity-Aware": "#10b981"}

print("✅ Setup complete")

✅ Setup complete


<a id="simulation"></a>
## 1. Strategy Execution
*Running parallel simulations across 3 routing methodologies.*

In [2]:
# Load data
trips_df = pd.read_csv(PROCESSED_DIR / 'trips_with_efficiency.csv')
drivers_df = pd.read_csv(PROCESSED_DIR / 'drivers.csv')

# Execute simulation
active_trips = trips_df[~trips_df['cancellation_reason'].notna()].copy()
simulation_results = run_simulation_comparison(active_trips, num_drivers=len(drivers_df))

display(simulation_results)

Running FCFS strategy...


Running Nearest-Driver strategy...


Running Capacity-Aware strategy...


,strategy,total_trips,on_time_rate,total_miles,avg_trip_duration,avg_idle_time,utilization_rate
0,FCFS,4853,40.30,106654.16,21.17,150.28,49.33
1,Nearest,4853,100.00,37996.71,21.17,2364.67,53.64
2,Capacity-Aware,4853,47.33,107620.70,21.17,219.49,68.49


<a id="finance"></a>
## 2. 💰 Financial Impact & ROI Projection
*Quantifying operational efficiency in dollars based on $2.15/mile NEMT industry standard.*

In [3]:
COST_PER_MILE = 2.15
results_df = simulation_results.copy()

# Calculate projected annual savings
baseline_miles = results_df[results_df['strategy'] == 'FCFS']['total_miles'].values[0]
results_df['total_cost'] = results_df['total_miles'] * COST_PER_MILE
results_df['savings_vs_baseline'] = (baseline_miles - results_df['total_miles']) * COST_PER_MILE
results_df['annual_projected_savings'] = results_df['savings_vs_baseline'] * 4 # Assume sample is 1 quarter

fig = px.bar(
    results_df, x='strategy', y='annual_projected_savings', 
    title='Projected Annual Cost Savings by Strategy (vs FCFS)',
    labels={'annual_projected_savings': 'Annual Savings ($)', 'strategy': 'Methodology'},
    color='strategy',
    color_discrete_map=STRATEGY_COLORS
)
fig.show()

<a id="findings"></a>
## 💡 Key Findings
**Strategic Recommendation for Operations**

In [4]:
best_strat = results_df.sort_values('on_time_rate', ascending=False).iloc[0]
utilization_gain = results_df['utilization_rate'].max() - results_df['utilization_rate'].min()

print("="*80)
print("🚀 ROUTING STRATEGY RECOMMENDATION")
print("="*80)
print(f"1. CHAMPION STRATEGY: The '{best_strat['strategy']}' methodology achieved the highest on-time rate ({best_strat['on_time_rate']:.1f}%).")
print(f"2. PROFITABILITY: Implementing Nearest-Driver logic projects a quarterly operational saving of ${results_df['savings_vs_baseline'].max():,.0f}.")
print(f"3. CAPACITY: Strategy tuning identified a {utilization_gain:.1f}% swing in vehicle utilization, suggesting current fleet size is optimized for load.")
print("="*80)

🚀 ROUTING STRATEGY RECOMMENDATION
1. CHAMPION STRATEGY: The 'Nearest' methodology achieved the highest on-time rate (100.0%).
2. PROFITABILITY: Implementing Nearest-Driver logic projects a quarterly operational saving of $147,614.
3. CAPACITY: Strategy tuning identified a 19.2% swing in vehicle utilization, suggesting current fleet size is optimized for load.
